In [2]:
import time
import signal 
import requests
from time import sleep
import pandas
import numpy

In [3]:
def calculate_atr(ticker, window=14):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/tas', params=payload)
    if not resp.ok:
        return None
    
    trades = resp.json()
    true_ranges = []

    for i in range(1, len(trades)):
        high = trades[i]['price']
        low = trades[i-1]['price']
        close = trades[i]['price']

        true_range = max(high - low, abs(high - close), abs(low - close))
        true_ranges.append(true_range)

    if len(true_ranges) < window:
        return None

    return sum(true_ranges[-window:]) / len(true_ranges[-window:])

def calculate_volatility(prices, window=10):
    if len(prices) < window:
        return None
    return sum([(p - sum(prices[-window:]) / window) ** 2 for p in prices[-window:]]) ** 0.5 / window

def adjust_risk_parameters(volatility):
    base_stop_loss = 0.02  # Minimum stop-loss percentage
    base_profit_target = 0.04  # Minimum profit-target percentage

    # Scale risk parameters with volatility
    stop_loss = base_stop_loss + (volatility * 0.5)  # Example scaling factor
    profit_target = base_profit_target + (volatility * 1.0)

    return stop_loss, profit_target


def decide_order_type(ticker, action, price, position):
    if ticker in ['CROW', 'DOVE']:  # Rebates for market orders
        order_type = 'MARKET'
    elif ticker in ['OWL', 'DUCK']:  # Rebates for limit orders
        order_type = 'LIMIT'
    else:
        order_type = 'LIMIT'  # Default to limit orders

    return {
        'ticker': ticker,
        'type': order_type,
        'quantity': ORDER_LIMIT,
        'price': price if order_type == 'LIMIT' else None,
        'action': action
    }

from math import gcd
from functools import reduce

def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def calculate_lcm(prices):
    # Scale prices to integers (e.g., handle up to 2 decimal places)
    scaled_prices = [int(price) for price in prices]  # Multiply by 100 to avoid floats
    lcm_scaled = reduce(lcm, scaled_prices)
    return lcm_scaled / 100  # Scale back to original form

    
def post_multiple_orders(ticker, action, total_quantity, price):
    remaining_quantity = total_quantity
    while remaining_quantity > 0:
        order_quantity = min(remaining_quantity, ORDER_LIMIT)
        s.post('http://localhost:9999/v1/orders', 
               params={'ticker': ticker, 'type': 'LIMIT', 
                       'quantity': order_quantity, 'price': price, 
                       'action': action})
        remaining_quantity -= order_quantity




In [7]:
s = requests.Session()
s.headers.update({'X-API-key': 'qqw'}) # Make sure you use YOUR API Key
ticker_list = ['OWL', 'CROW', 'DOVE', 'DUCK']

# global variables
TARGET_POSITIONS = {
    'OWL': 7700,
    'DOVE': 30800,
    'CROW': 11200,
    'DUCK': 17600
}

max_long_exposure = 40000
MAX_SHORT_EXPOSURE = 0
ORDER_LIMIT = 500
# Risk Management Parameters
STOP_LOSS_PERCENT = 0.05  # Exit position if price drops 5% below entry
PROFIT_TARGET_PERCENT = 0.10  # Exit position if price rises 10% above entry

# Store entry prices for each ticker
entry_prices = {}


def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
        
def price():
    prices = {}
    for ticker in ticker_list:
        resp = s.get(f'http://localhost:9999/v1/securities/tas?ticker={ticker}')
        if resp.ok:
            response = resp.json()
            if response:  # Ensure there's at least one trade
                prices[ticker] = response[0]['price']
                print(f'{ticker}:', prices[ticker])
            else:
                prices[ticker] = None  # No recent trades
        else:
            prices[ticker] = None  # Handle failed request
    return prices


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        securities = resp.json()
        for security in securities:
            if security['ticker'] == ticker:
                return security['position']
    return 0  # Return 0 if ticker not found or request fails


def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders

def get_order_status(order_id):
    resp = s.get ('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']

def main():
    import logging
    from concurrent.futures import ThreadPoolExecutor
    from time import sleep

    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

    # Define tickers and their respective MAX_LONG_EXPOSURE
    ticker_limits = {
        'OWL': 7700,
        'DOVE': 30800,
        'CROW': 11200,
        'DUCK': 15000
    }

    # Track sell states to prevent re-buying after selling
    sell_flags = {ticker: False for ticker in ticker_limits.keys()}
    
    ticker_list = list(ticker_limits.keys())
    tick, status = get_tick()

    # Main loop to process each ticker while the market is active
    def process_ticker(ticker_symbol):
        max_long_exposure = ticker_limits[ticker_symbol]  # Get the limit for this ticker
        position = get_position(ticker_symbol)  # Fetch current position for this ticker
        best_bid_price, best_ask_price = get_bid_ask(ticker_symbol)
        current_prices = price()  # Fetch current prices for all tickers
        current_price = current_prices.get(ticker_symbol, None)

        # Ensure we have a valid entry price or set it if not already recorded
        if ticker_symbol not in entry_prices and current_price:
            entry_prices[ticker_symbol] = current_price

        entry_price = entry_prices.get(ticker_symbol, None)

        # Selling logic: Exit position if profit targets are met
        if entry_price and current_price:
            if position > MAX_SHORT_EXPOSURE:
                # If price has increased by 7% or more, sell
                if current_price >= entry_price * 1.07:
                    logging.info(f"Placing SELL order for {ticker_symbol} at {best_ask_price} (7% profit target reached)")
                    resp = s.post(
                        'http://localhost:9999/v1/orders',
                        params={
                            'ticker': ticker_symbol,
                            'type': 'LIMIT',
                            'quantity': ORDER_LIMIT,
                            'price': best_ask_price,
                            'action': 'SELL'
                        }
                    )
                    if resp.status_code == 200:
                        logging.info(f"SELL order placed successfully: {resp.json()}")
                        sell_flags[ticker_symbol] = True  # Mark ticker as sold
                    else:
                        logging.error(f"Failed to place SELL order: {resp.status_code} - {resp.text}")
                # If price has increased by 5% or more, sell
                elif current_price >= entry_price * 1.05:
                    logging.info(f"Placing SELL order for {ticker_symbol} at {best_ask_price} (5% profit target reached)")
                    resp = s.post(
                        'http://localhost:9999/v1/orders',
                        params={
                            'ticker': ticker_symbol,
                            'type': 'LIMIT',
                            'quantity': ORDER_LIMIT,
                            'price': best_ask_price,
                            'action': 'SELL'
                        }
                    )
                    if resp.status_code == 200:
                        logging.info(f"SELL order placed successfully: {resp.json()}")
                        sell_flags[ticker_symbol] = True  # Mark ticker as sold
                    else:
                        logging.error(f"Failed to place SELL order: {resp.status_code} - {resp.text}")

        # Buying logic: Only buy if the sell flag is False
        if not sell_flags[ticker_symbol] and position < max_long_exposure:
            logging.info(f"Placing BUY order for {ticker_symbol} at {best_bid_price}")
            resp = s.post(
                'http://localhost:9999/v1/orders',
                params={
                    'ticker': ticker_symbol,
                    'type': 'LIMIT',
                    'quantity': ORDER_LIMIT,
                    'price': best_bid_price,
                    'action': 'BUY'
                }
            )
            if resp.status_code == 200:
                logging.info(f"BUY order placed successfully: {resp.json()}")
            else:
                logging.error(f"Failed to place BUY order: {resp.status_code} - {resp.text}")

        # Allow some time for orders to process
        sleep(0.75)

        # Cancel all open orders for the ticker
        logging.info(f"Cancelling all orders for {ticker_symbol}")
        s.post('http://localhost:9999/v1/commands/cancel', params={'ticker': ticker_symbol})

    # Main loop to keep processing while the market is active
    while status == 'ACTIVE':
        with ThreadPoolExecutor(max_workers=4) as executor:
            executor.map(process_ticker, ticker_list)

        tick, status = get_tick()

    logging.info("Market is no longer active. Stopping trading operations.")



if __name__ == '__main__':
    main()


2024-11-29 09:03:05,943 - Placing BUY order for DOVE at 27.53
2024-11-29 09:03:05,952 - Placing BUY order for DUCK at 76.01
2024-11-29 09:03:05,972 - BUY order placed successfully: {'order_id': 13783, 'period': 1, 'tick': 144, 'trader_id': 'SMITH1-1', 'ticker': 'DOVE', 'quantity': 500.0, 'price': 27.53, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}
2024-11-29 09:03:05,999 - BUY order placed successfully: {'order_id': 13786, 'period': 1, 'tick': 144, 'trader_id': 'SMITH1-1', 'ticker': 'DUCK', 'quantity': 500.0, 'price': 76.01, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}


OWL: 171.4
OWL: 171.4
OWL: 171.4
OWL: 171.4
CROW: 110.73
CROW: 110.73
CROW: 110.73
CROW: 110.73
DOVE: 27.57
DOVE: 27.57
DOVE: 27.57
DOVE: 27.57
DUCK: 76.01
DUCK: 76.01
DUCK: 76.01
DUCK: 76.01


2024-11-29 09:03:06,682 - Cancelling all orders for OWL
2024-11-29 09:03:06,689 - Cancelling all orders for CROW
2024-11-29 09:03:06,728 - Cancelling all orders for DOVE
2024-11-29 09:03:06,750 - Cancelling all orders for DUCK
2024-11-29 09:03:06,880 - Placing BUY order for DOVE at 27.53
2024-11-29 09:03:06,887 - Placing BUY order for DUCK at 76.37
2024-11-29 09:03:06,927 - BUY order placed successfully: {'order_id': 13870, 'period': 1, 'tick': 145, 'trader_id': 'SMITH1-1', 'ticker': 'DOVE', 'quantity': 500.0, 'price': 27.53, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}
2024-11-29 09:03:06,948 - BUY order placed successfully: {'order_id': 13874, 'period': 1, 'tick': 145, 'trader_id': 'SMITH1-1', 'ticker': 'DUCK', 'quantity': 500.0, 'price': 76.37, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}


OWL: 171.75
OWL: 171.75
OWL: 171.75
OWL: 171.75
CROW: 110.66
CROW: 110.66
DOVE: 27.63
CROW: 110.66
CROW: 110.66
DOVE: 27.63
DOVE: 27.63
DOVE: 27.63
DUCK: 76.37
DUCK: 76.37
DUCK: 76.37
DUCK: 76.37


2024-11-29 09:03:07,616 - Cancelling all orders for OWL
2024-11-29 09:03:07,623 - Cancelling all orders for CROW
2024-11-29 09:03:07,678 - Cancelling all orders for DOVE
2024-11-29 09:03:07,700 - Cancelling all orders for DUCK
2024-11-29 09:03:07,803 - Placing BUY order for DOVE at 27.67
2024-11-29 09:03:07,839 - BUY order placed successfully: {'order_id': 13956, 'period': 1, 'tick': 146, 'trader_id': 'SMITH1-1', 'ticker': 'DOVE', 'quantity': 500.0, 'price': 27.67, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}
2024-11-29 09:03:07,839 - Placing BUY order for DUCK at 76.13
2024-11-29 09:03:07,868 - BUY order placed successfully: {'order_id': 13960, 'period': 1, 'tick': 146, 'trader_id': 'SMITH1-1', 'ticker': 'DUCK', 'quantity': 500.0, 'price': 76.13, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}


...

OWL:OWL: 172.04
 172.04
OWL: 172.04
OWL: 172.04
CROW: 110.23
CROW: 110.23
DOVE: 28.24
CROW: 110.23
CROW: 110.23
DOVE: 28.24
DOVE: 28.24
DUCK: 75.45
DOVE: 28.24
DUCK: 75.45
DUCK: 75.44
DUCK: 75.44


2024-11-29 09:03:17,958 - Cancelling all orders for OWL
2024-11-29 09:03:17,965 - Cancelling all orders for CROW
2024-11-29 09:03:17,999 - Cancelling all orders for DUCK
2024-11-29 09:03:18,020 - Cancelling all orders for DOVE
2024-11-29 09:03:18,138 - Placing BUY order for DOVE at 28.35
2024-11-29 09:03:18,143 - Placing BUY order for DUCK at 75.32
2024-11-29 09:03:18,178 - BUY order placed successfully: {'order_id': 14915, 'period': 1, 'tick': 157, 'trader_id': 'SMITH1-1', 'ticker': 'DOVE', 'quantity': 500.0, 'price': 28.35, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 0.0, 'vwap': None, 'status': 'OPEN'}
2024-11-29 09:03:18,198 - BUY order placed successfully: {'order_id': 14918, 'period': 1, 'tick': 157, 'trader_id': 'SMITH1-1', 'ticker': 'DUCK', 'quantity': 500.0, 'price': 75.32, 'type': 'LIMIT', 'action': 'BUY', 'quantity_filled': 500.0, 'vwap': 75.31, 'status': 'OPEN'}


OWL: 171.82
OWL: 171.82
OWL: 171.82
OWL: 171.82
CROW: 110.02
CROW: 110.02
DOVE: 28.31
CROW: 110.02
CROW: 110.02
DOVE: 28.31
DOVE: 28.31
DUCK: 75.32
DOVE: 28.31
DUCK: 75.32
DUCK: 75.32
DUCK: 75.23


2024-11-29 09:03:18,877 - Cancelling all orders for OWL
2024-11-29 09:03:18,899 - Cancelling all orders for CROW
2024-11-29 09:03:18,935 - Cancelling all orders for DOVE
2024-11-29 09:03:18,957 - Cancelling all orders for DUCK


TypeError: cannot unpack non-iterable NoneType object